# cmems

## 散点图

In [6]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os

plt.rcParams['font.sans-serif'] = ['SimHei']

def plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date, output_dir):
    """
    绘制订正前、订正后的非参考数据与参考数据的散点图
    :param non_ref_data: 订正前的非参考数据 (2D numpy array)
    :param corrected_data: 订正后的非参考数据 (2D numpy array)
    :param ref_data: 参考数据 (2D numpy array)
    :param date: 当前日期 (str)
    :param output_dir: 输出散点图的保存目录
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 平铺数组，去除 NaN 数据
    mask = (~np.isnan(non_ref_data)) & (~np.isnan(ref_data))
    x_non_ref = non_ref_data[mask]
    x_corrected = corrected_data[mask]
    y_ref = ref_data[mask]
    
    # 绘制订正前的散点图
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.scatter(x_non_ref, y_ref, s=2, alpha=0.5, color='blue')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
   
    plt.xlabel("非参考数据 (订正前)")
    plt.ylabel("参考数据")
    plt.title(f"订正前 ({date})")
    
    # 绘制订正后的散点图
    plt.subplot(1, 2, 2)
    plt.scatter(x_corrected, y_ref, s=2, alpha=0.5, color='red')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
    plt.xlabel("非参考数据 (订正后)")
    plt.ylabel("参考数据")
    plt.title(f"订正后 ({date})")
    
    # 保存散点图
    plt.tight_layout()
    output_file = os.path.join(output_dir, f"scatter_comparison_{date}.png")
    plt.savefig(output_file, dpi=100)
    plt.close()
    print(f"Saved scatter plot: {output_file}")

def generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date):
    """
    生成非参考数据的订正前后与参考数据的散点图
    :param input_dir: 订正前非参考数据目录
    :param corrected_dir: 订正后数据目录
    :param reference_dir: 参考数据目录
    :param output_dir: 输出散点图的保存目录
    :param hemisphere: 'north' 或 'south'
    :param start_date: 开始日期 (str)
    :param end_date: 结束日期 (str)
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)

    for date in date_range:
        date_str = date.strftime('%Y%m%d')
        print(f"Processing scatter plot for date: {date_str}")

        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_{hemisphere}_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"cmems_pdfcorrected_{hemisphere}_SIT_25000_{date_str}.nc")
        non_ref_file = os.path.join(input_dir, f"cmems_{hemisphere}_SIT_25000_{date_str}.nc")

        # 检查文件是否存在
        if not all(os.path.exists(f) for f in [ref_file, corrected_file, non_ref_file]):
            print(f"Missing file for date {date_str}, skipping...")
            continue

        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['cmems_sit'].values
        non_ref_data = xr.open_dataset(non_ref_file)['cmems_sit'].values

        # 绘制散点图
        plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date_str, output_dir)

if __name__ == "__main__":
    import pandas as pd

    # 文件目录路径
    input_dir = r"D:\zmh\icethickness\Copernicus-Global\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\Copernicus-Global\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\Copernicus-Global\2_scatter_plots"
    # 若output_dir不存在，创建output_dir
    os.makedirs(output_dir, exist_ok=True)

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"

    generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date)


Processing scatter plot for date: 20220101
Missing file for date 20220101, skipping...
Processing scatter plot for date: 20220102
Saved scatter plot: D:\zmh\icethickness\Copernicus-Global\2_scatter_plots\scatter_comparison_20220102.png
Processing scatter plot for date: 20220103
Saved scatter plot: D:\zmh\icethickness\Copernicus-Global\2_scatter_plots\scatter_comparison_20220103.png
Processing scatter plot for date: 20220104
Saved scatter plot: D:\zmh\icethickness\Copernicus-Global\2_scatter_plots\scatter_comparison_20220104.png
Processing scatter plot for date: 20220105
Saved scatter plot: D:\zmh\icethickness\Copernicus-Global\2_scatter_plots\scatter_comparison_20220105.png
Processing scatter plot for date: 20220106
Missing file for date 20220106, skipping...
Processing scatter plot for date: 20220107
Missing file for date 20220107, skipping...
Processing scatter plot for date: 20220108
Saved scatter plot: D:\zmh\icethickness\Copernicus-Global\2_scatter_plots\scatter_comparison_2022010

## 偏差

In [7]:
import numpy as np
import xarray as xr
import os
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import pandas as pd 

plt.rcParams['font.sans-serif'] = ['SimHei']

def calculate_metrics(reference, corrected, original):
    """
    计算均方根误差（RMSE）、偏差（Bias） 和相关系数（CC）
    """
    # 有效数据掩模
    mask = ~np.isnan(reference) & ~np.isnan(corrected) & ~np.isnan(original)
    ref = reference[mask]
    corr = corrected[mask]
    orig = original[mask]
    
    # 计算 RMSE、Bias 和相关系数
    rmse_corrected = np.sqrt(mean_squared_error(ref, corr))
    rmse_original = np.sqrt(mean_squared_error(ref, orig))
    bias_corrected = np.mean(corr - ref)
    bias_original = np.mean(orig - ref)
    cc_corrected, _ = pearsonr(ref, corr)
    cc_original, _ = pearsonr(ref, orig)
    
    return {
        'RMSE (Corrected)': rmse_corrected,
        'RMSE (Original)': rmse_original,
        'Bias (Corrected)': bias_corrected,
        'Bias (Original)': bias_original,
        'CC (Corrected)': cc_corrected,
        'CC (Original)': cc_original
    }

def compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date):
    """
    比较PDF订正后的结果和原始数据与参考数据之间的差异
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)
    results = []
    
    for current_date in date_range:
        date_str = current_date.strftime('%Y%m%d')
        print(f"Processing date: {date_str}")
        
        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_north_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"cmems_pdfcorrected_north_SIT_25000_{date_str}.nc")
        original_file = os.path.join(input_dir, f"cmems_north_SIT_25000_{date_str}.nc")
        
        if not all(map(os.path.exists, [ref_file, corrected_file, original_file])):
            print(f"Skipping {date_str}, missing file.")
            continue
        
        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['cmems_sit'].values
        original_data = xr.open_dataset(original_file)['cmems_sit'].values
        
        # 计算指标
        metrics = calculate_metrics(ref_data, corrected_data, original_data)
        metrics['Date'] = date_str
        results.append(metrics)
    
    # 保存结果为CSV文件
    results_df = pd.DataFrame(results)
    output_file = os.path.join(output_dir, 'comparison_metrics_north_cmems.csv')
    results_df.to_csv(output_file, index=False)
    print(f"Saved metrics to {output_file}")
    

if __name__ == "__main__":
    # 参数设置

    original_dir = r"D:\zmh\icethickness\Copernicus-Global\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\Copernicus-Global\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\Copernicus-Global\3_bias"

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"
    
    compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date)


Processing date: 20220101
Skipping 20220101, missing file.
Processing date: 20220102
Processing date: 20220103
Processing date: 20220104
Processing date: 20220105
Processing date: 20220106
Skipping 20220106, missing file.
Processing date: 20220107
Skipping 20220107, missing file.
Processing date: 20220108
Processing date: 20220109
Processing date: 20220110
Processing date: 20220111
Processing date: 20220112
Processing date: 20220113
Processing date: 20220114
Processing date: 20220115
Skipping 20220115, missing file.
Processing date: 20220116
Processing date: 20220117
Skipping 20220117, missing file.
Processing date: 20220118
Skipping 20220118, missing file.
Processing date: 20220119
Processing date: 20220120
Processing date: 20220121
Processing date: 20220122
Processing date: 20220123
Processing date: 20220124
Processing date: 20220125
Processing date: 20220126
Processing date: 20220127
Processing date: 20220128
Processing date: 20220129
Processing date: 20220130
Processing date: 20220

C:\Users\holo\AppData\Local\Temp\ipykernel_15804\4139987018.py:25: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_corrected, _ = pearsonr(ref, corr)
C:\Users\holo\AppData\Local\Temp\ipykernel_15804\4139987018.py:26: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_original, _ = pearsonr(ref, orig)


Processing date: 20221116
Processing date: 20221117
Processing date: 20221118
Processing date: 20221119
Processing date: 20221120
Processing date: 20221121
Processing date: 20221122
Processing date: 20221123
Processing date: 20221124
Processing date: 20221125
Processing date: 20221126
Processing date: 20221127
Processing date: 20221128
Processing date: 20221129
Processing date: 20221130
Processing date: 20221201
Processing date: 20221202
Processing date: 20221203
Processing date: 20221204
Processing date: 20221205
Processing date: 20221206
Processing date: 20221207
Processing date: 20221208
Processing date: 20221209
Processing date: 20221210
Processing date: 20221211
Processing date: 20221212
Processing date: 20221213
Processing date: 20221214
Processing date: 20221215
Processing date: 20221216
Processing date: 20221217
Processing date: 20221218
Skipping 20221218, missing file.
Processing date: 20221219
Processing date: 20221220
Skipping 20221220, missing file.
Processing date: 2022122

# CS2

## 散点图

In [8]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os

plt.rcParams['font.sans-serif'] = ['SimHei']

def plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date, output_dir):
    """
    绘制订正前、订正后的非参考数据与参考数据的散点图
    :param non_ref_data: 订正前的非参考数据 (2D numpy array)
    :param corrected_data: 订正后的非参考数据 (2D numpy array)
    :param ref_data: 参考数据 (2D numpy array)
    :param date: 当前日期 (str)
    :param output_dir: 输出散点图的保存目录
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 平铺数组，去除 NaN 数据
    mask = (~np.isnan(non_ref_data)) & (~np.isnan(ref_data))
    x_non_ref = non_ref_data[mask]
    x_corrected = corrected_data[mask]
    y_ref = ref_data[mask]
    
    # 绘制订正前的散点图
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.scatter(x_non_ref, y_ref, s=2, alpha=0.5, color='blue')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
   
    plt.xlabel("非参考数据 (订正前)")
    plt.ylabel("参考数据")
    plt.title(f"订正前 ({date})")
    
    # 绘制订正后的散点图
    plt.subplot(1, 2, 2)
    plt.scatter(x_corrected, y_ref, s=2, alpha=0.5, color='red')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
    plt.xlabel("非参考数据 (订正后)")
    plt.ylabel("参考数据")
    plt.title(f"订正后 ({date})")
    
    # 保存散点图
    plt.tight_layout()
    output_file = os.path.join(output_dir, f"scatter_comparison_{date}.png")
    plt.savefig(output_file, dpi=100)
    plt.close()
    print(f"Saved scatter plot: {output_file}")

def generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date):
    """
    生成非参考数据的订正前后与参考数据的散点图
    :param input_dir: 订正前非参考数据目录
    :param corrected_dir: 订正后数据目录
    :param reference_dir: 参考数据目录
    :param output_dir: 输出散点图的保存目录
    :param hemisphere: 'north' 或 'south'
    :param start_date: 开始日期 (str)
    :param end_date: 结束日期 (str)
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)

    for date in date_range:
        date_str = date.strftime('%Y%m%d')
        print(f"Processing scatter plot for date: {date_str}")

        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_{hemisphere}_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"CS2_pdfcorrected_{hemisphere}_SIT_25000_{date_str}.nc")
        non_ref_file = os.path.join(input_dir, f"CS2_{hemisphere}_SIT_25000_{date_str}.nc")

        # 检查文件是否存在
        if not all(os.path.exists(f) for f in [ref_file, corrected_file, non_ref_file]):
            print(f"Missing file for date {date_str}, skipping...")
            continue

        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['CS2_sit'].values
        non_ref_data = xr.open_dataset(non_ref_file)['CS2_sit'].values

        # 绘制散点图
        plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date_str, output_dir)

if __name__ == "__main__":
    import pandas as pd

    # 文件目录路径
    input_dir = r"D:\zmh\icethickness\CS2\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\CS2\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\CS2\2_scatter_plots"
    # 若output_dir不存在，创建output_dir
    os.makedirs(output_dir, exist_ok=True)

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"

    generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date)

Processing scatter plot for date: 20220101
Missing file for date 20220101, skipping...
Processing scatter plot for date: 20220102
Saved scatter plot: D:\zmh\icethickness\CS2\2_scatter_plots\scatter_comparison_20220102.png
Processing scatter plot for date: 20220103
Saved scatter plot: D:\zmh\icethickness\CS2\2_scatter_plots\scatter_comparison_20220103.png
Processing scatter plot for date: 20220104
Saved scatter plot: D:\zmh\icethickness\CS2\2_scatter_plots\scatter_comparison_20220104.png
Processing scatter plot for date: 20220105
Saved scatter plot: D:\zmh\icethickness\CS2\2_scatter_plots\scatter_comparison_20220105.png
Processing scatter plot for date: 20220106
Missing file for date 20220106, skipping...
Processing scatter plot for date: 20220107
Missing file for date 20220107, skipping...
Processing scatter plot for date: 20220108
Saved scatter plot: D:\zmh\icethickness\CS2\2_scatter_plots\scatter_comparison_20220108.png
Processing scatter plot for date: 20220109
Saved scatter plot: D

## 偏差

In [9]:
import numpy as np
import xarray as xr
import os
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import pandas as pd 

plt.rcParams['font.sans-serif'] = ['SimHei']

def calculate_metrics(reference, corrected, original):
    """
    计算均方根误差（RMSE）、偏差（Bias） 和相关系数（CC）
    """
    # 有效数据掩模
    mask = ~np.isnan(reference) & ~np.isnan(corrected) & ~np.isnan(original)
    ref = reference[mask]
    corr = corrected[mask]
    orig = original[mask]
    
    # 计算 RMSE、Bias 和相关系数
    rmse_corrected = np.sqrt(mean_squared_error(ref, corr))
    rmse_original = np.sqrt(mean_squared_error(ref, orig))
    bias_corrected = np.mean(corr - ref)
    bias_original = np.mean(orig - ref)
    cc_corrected, _ = pearsonr(ref, corr)
    cc_original, _ = pearsonr(ref, orig)
    
    return {
        'RMSE (Corrected)': rmse_corrected,
        'RMSE (Original)': rmse_original,
        'Bias (Corrected)': bias_corrected,
        'Bias (Original)': bias_original,
        'CC (Corrected)': cc_corrected,
        'CC (Original)': cc_original
    }

def compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date):
    """
    比较PDF订正后的结果和原始数据与参考数据之间的差异
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)
    results = []
    
    for current_date in date_range:
        date_str = current_date.strftime('%Y%m%d')
        print(f"Processing date: {date_str}")
        
        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_north_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"CS2_pdfcorrected_north_SIT_25000_{date_str}.nc")
        original_file = os.path.join(input_dir, f"CS2_north_SIT_25000_{date_str}.nc")
        
        if not all(map(os.path.exists, [ref_file, corrected_file, original_file])):
            print(f"Skipping {date_str}, missing file.")
            continue
        
        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['CS2_sit'].values
        original_data = xr.open_dataset(original_file)['CS2_sit'].values
        
        # 计算指标
        metrics = calculate_metrics(ref_data, corrected_data, original_data)
        metrics['Date'] = date_str
        results.append(metrics)
    
    # 保存结果为CSV文件
    results_df = pd.DataFrame(results)
    output_file = os.path.join(output_dir, 'comparison_metrics_north_CS2.csv')
    results_df.to_csv(output_file, index=False)
    print(f"Saved metrics to {output_file}")
    

if __name__ == "__main__":
    # 参数设置

    original_dir = r"D:\zmh\icethickness\CS2\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\CS2\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\CS2\3_bias"

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"
    
    compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date)


Processing date: 20220101
Skipping 20220101, missing file.
Processing date: 20220102
Processing date: 20220103
Processing date: 20220104
Processing date: 20220105
Processing date: 20220106
Skipping 20220106, missing file.
Processing date: 20220107
Skipping 20220107, missing file.
Processing date: 20220108
Processing date: 20220109
Processing date: 20220110
Processing date: 20220111
Processing date: 20220112
Processing date: 20220113
Processing date: 20220114
Processing date: 20220115
Skipping 20220115, missing file.
Processing date: 20220116
Processing date: 20220117
Skipping 20220117, missing file.
Processing date: 20220118
Skipping 20220118, missing file.
Processing date: 20220119
Processing date: 20220120
Processing date: 20220121
Processing date: 20220122
Processing date: 20220123
Processing date: 20220124
Processing date: 20220125
Processing date: 20220126
Processing date: 20220127
Processing date: 20220128
Processing date: 20220129
Processing date: 20220130
Processing date: 20220

C:\Users\holo\AppData\Local\Temp\ipykernel_15804\1954144531.py:25: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_corrected, _ = pearsonr(ref, corr)
C:\Users\holo\AppData\Local\Temp\ipykernel_15804\1954144531.py:26: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_original, _ = pearsonr(ref, orig)


Processing date: 20221125
Processing date: 20221126
Processing date: 20221127
Processing date: 20221128
Processing date: 20221129
Processing date: 20221130
Processing date: 20221201
Processing date: 20221202
Processing date: 20221203
Processing date: 20221204
Processing date: 20221205
Processing date: 20221206
Processing date: 20221207
Processing date: 20221208
Processing date: 20221209
Processing date: 20221210
Processing date: 20221211
Processing date: 20221212
Processing date: 20221213
Processing date: 20221214
Processing date: 20221215
Processing date: 20221216
Processing date: 20221217
Processing date: 20221218
Skipping 20221218, missing file.
Processing date: 20221219
Processing date: 20221220
Skipping 20221220, missing file.
Processing date: 20221221
Processing date: 20221222
Processing date: 20221223
Skipping 20221223, missing file.
Processing date: 20221224
Processing date: 20221225
Processing date: 20221226
Processing date: 20221227
Processing date: 20221228
Skipping 20221228

# rdeft4

## 散点图

In [6]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os

plt.rcParams['font.sans-serif'] = ['SimHei']

def plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date, output_dir):
    """
    绘制订正前、订正后的非参考数据与参考数据的散点图
    :param non_ref_data: 订正前的非参考数据 (2D numpy array)
    :param corrected_data: 订正后的非参考数据 (2D numpy array)
    :param ref_data: 参考数据 (2D numpy array)
    :param date: 当前日期 (str)
    :param output_dir: 输出散点图的保存目录
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 平铺数组，去除 NaN 数据
    mask = (~np.isnan(non_ref_data)) & (~np.isnan(ref_data))
    x_non_ref = non_ref_data[mask]
    x_corrected = corrected_data[mask]
    y_ref = ref_data[mask]
    
    # 绘制订正前的散点图
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.scatter(x_non_ref, y_ref, s=2, alpha=0.5, color='blue')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
   
    plt.xlabel("非参考数据 (订正前)")
    plt.ylabel("参考数据")
    plt.title(f"订正前 ({date})")
    
    # 绘制订正后的散点图
    plt.subplot(1, 2, 2)
    plt.scatter(x_corrected, y_ref, s=2, alpha=0.5, color='red')
    plt.plot([0, 3], [0, 3], 'k--', lw=1)  # 1:1 线
    plt.xlim(0, 3)
    plt.ylim(0, 3)
    plt.xlabel("非参考数据 (订正后)")
    plt.ylabel("参考数据")
    plt.title(f"订正后 ({date})")
    
    # 保存散点图
    plt.tight_layout()
    output_file = os.path.join(output_dir, f"scatter_comparison_{date}.png")
    plt.savefig(output_file, dpi=100)
    plt.close()
    print(f"Saved scatter plot: {output_file}")

def generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date):
    """
    生成非参考数据的订正前后与参考数据的散点图
    :param input_dir: 订正前非参考数据目录
    :param corrected_dir: 订正后数据目录
    :param reference_dir: 参考数据目录
    :param output_dir: 输出散点图的保存目录
    :param hemisphere: 'north' 或 'south'
    :param start_date: 开始日期 (str)
    :param end_date: 结束日期 (str)
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)

    for date in date_range:
        date_str = date.strftime('%Y%m%d')
        print(f"Processing scatter plot for date: {date_str}")

        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_{hemisphere}_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"rdeft4_pdfcorrected_{hemisphere}_SIT_25000_{date_str}.nc")
        non_ref_file = os.path.join(input_dir, f"rdeft4_{hemisphere}_SIT_25000_{date_str}.nc")

        # 检查文件是否存在
        if not all(os.path.exists(f) for f in [ref_file, corrected_file, non_ref_file]):
            print(f"Missing file for date {date_str}, skipping...")
            continue

        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['rdeft4_sit'].values
        non_ref_data = xr.open_dataset(non_ref_file)['rdeft4_sit'].values

        # 绘制散点图
        plot_scatter_comparison(non_ref_data, corrected_data, ref_data, date_str, output_dir)

if __name__ == "__main__":
    import pandas as pd

    # 文件目录路径
    input_dir = r"D:\zmh\icethickness\NSIDC-R\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\NSIDC-R\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\NSIDC-R\2_scatter_plots"
    os.makedirs(output_dir, exist_ok=True)

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"

    generate_scatter_plots(input_dir, corrected_dir, reference_dir, output_dir, hemisphere, start_date, end_date)

Processing scatter plot for date: 20220101
Missing file for date 20220101, skipping...
Processing scatter plot for date: 20220102
Saved scatter plot: D:\zmh\icethickness\NSIDC-R\2_scatter_plots\scatter_comparison_20220102.png
Processing scatter plot for date: 20220103
Saved scatter plot: D:\zmh\icethickness\NSIDC-R\2_scatter_plots\scatter_comparison_20220103.png
Processing scatter plot for date: 20220104
Saved scatter plot: D:\zmh\icethickness\NSIDC-R\2_scatter_plots\scatter_comparison_20220104.png
Processing scatter plot for date: 20220105
Saved scatter plot: D:\zmh\icethickness\NSIDC-R\2_scatter_plots\scatter_comparison_20220105.png
Processing scatter plot for date: 20220106
Missing file for date 20220106, skipping...
Processing scatter plot for date: 20220107
Missing file for date 20220107, skipping...
Processing scatter plot for date: 20220108
Saved scatter plot: D:\zmh\icethickness\NSIDC-R\2_scatter_plots\scatter_comparison_20220108.png
Processing scatter plot for date: 20220109
S

## 偏差

In [8]:
import numpy as np
import xarray as xr
import os
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import pandas as pd 

plt.rcParams['font.sans-serif'] = ['SimHei']

def calculate_metrics(reference, corrected, original):
    """
    计算均方根误差（RMSE）、偏差（Bias） 和相关系数（CC）
    """
    # 有效数据掩模
    mask = ~np.isnan(reference) & ~np.isnan(corrected) & ~np.isnan(original)
    ref = reference[mask]
    corr = corrected[mask]
    orig = original[mask]
    
    # 计算 RMSE、Bias 和相关系数
    rmse_corrected = np.sqrt(mean_squared_error(ref, corr))
    rmse_original = np.sqrt(mean_squared_error(ref, orig))
    bias_corrected = np.mean(corr - ref)
    bias_original = np.mean(orig - ref)
    cc_corrected, _ = pearsonr(ref, corr)
    cc_original, _ = pearsonr(ref, orig)
    
    return {
        'RMSE (Corrected)': rmse_corrected,
        'RMSE (Original)': rmse_original,
        'Bias (Corrected)': bias_corrected,
        'Bias (Original)': bias_original,
        'CC (Corrected)': cc_corrected,
        'CC (Original)': cc_original
    }

def compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date):
    """
    比较PDF订正后的结果和原始数据与参考数据之间的差异
    """
    os.makedirs(output_dir, exist_ok=True)
    date_range = pd.date_range(start=start_date, end=end_date)
    results = []
    
    for current_date in date_range:
        date_str = current_date.strftime('%Y%m%d')
        print(f"Processing date: {date_str}")
        
        # 文件路径
        ref_file = os.path.join(reference_dir, f"APPX_north_SIT_25000_{date_str}.nc")
        corrected_file = os.path.join(corrected_dir, f"rdeft4_pdfcorrected_north_SIT_25000_{date_str}.nc")
        original_file = os.path.join(input_dir, f"rdeft4_north_SIT_25000_{date_str}.nc")
        
        if not all(map(os.path.exists, [ref_file, corrected_file, original_file])):
            print(f"Skipping {date_str}, missing file.")
            continue
        
        # 读取数据
        ref_data = xr.open_dataset(ref_file)['appx_sit'].values
        corrected_data = xr.open_dataset(corrected_file)['rdeft4_sit'].values
        original_data = xr.open_dataset(original_file)['rdeft4_sit'].values
        
        # 计算指标
        metrics = calculate_metrics(ref_data, corrected_data, original_data)
        metrics['Date'] = date_str
        results.append(metrics)
    
    # 保存结果为CSV文件
    results_df = pd.DataFrame(results)
    output_file = os.path.join(output_dir, 'comparison_metrics_north_rdeft4.csv')
    results_df.to_csv(output_file, index=False)
    print(f"Saved metrics to {output_file}")
    

if __name__ == "__main__":
    # 参数设置

    original_dir = r"D:\zmh\icethickness\NSIDC-R\0_pre_com\north"
    corrected_dir = r"D:\zmh\icethickness\NSIDC-R\1_PDFcorrected\north"
    reference_dir = r"D:\zmh\icethickness\APP-X\0_pre_com\north"
    output_dir = r"D:\zmh\icethickness\NSIDC-R\3_bias"

    hemisphere = "north"
    start_date = "2022-01-01"
    end_date = "2022-12-31"
    
    compare_datasets(reference_dir, corrected_dir, original_dir, output_dir, hemisphere, start_date, end_date)


Processing date: 20220101
Skipping 20220101, missing file.
Processing date: 20220102
Processing date: 20220103
Processing date: 20220104
Processing date: 20220105
Processing date: 20220106
Skipping 20220106, missing file.
Processing date: 20220107
Skipping 20220107, missing file.
Processing date: 20220108
Processing date: 20220109
Processing date: 20220110
Processing date: 20220111
Processing date: 20220112
Processing date: 20220113
Processing date: 20220114
Processing date: 20220115
Skipping 20220115, missing file.
Processing date: 20220116
Processing date: 20220117
Skipping 20220117, missing file.
Processing date: 20220118
Skipping 20220118, missing file.
Processing date: 20220119
Processing date: 20220120
Processing date: 20220121
Processing date: 20220122
Processing date: 20220123
Processing date: 20220124
Processing date: 20220125
Processing date: 20220126
Processing date: 20220127
Processing date: 20220128
Processing date: 20220129
Processing date: 20220130
Processing date: 20220

C:\Users\holo\AppData\Local\Temp\ipykernel_22220\1936077561.py:25: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_corrected, _ = pearsonr(ref, corr)
C:\Users\holo\AppData\Local\Temp\ipykernel_22220\1936077561.py:26: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cc_original, _ = pearsonr(ref, orig)


Processing date: 20221120
Processing date: 20221121
Processing date: 20221122
Processing date: 20221123
Processing date: 20221124
Processing date: 20221125
Processing date: 20221126
Processing date: 20221127
Processing date: 20221128
Processing date: 20221129
Processing date: 20221130
Processing date: 20221201
Processing date: 20221202
Processing date: 20221203
Processing date: 20221204
Processing date: 20221205
Processing date: 20221206
Processing date: 20221207
Processing date: 20221208
Processing date: 20221209
Processing date: 20221210
Processing date: 20221211
Processing date: 20221212
Processing date: 20221213
Processing date: 20221214
Processing date: 20221215
Processing date: 20221216
Processing date: 20221217
Processing date: 20221218
Skipping 20221218, missing file.
Processing date: 20221219
Processing date: 20221220
Skipping 20221220, missing file.
Processing date: 20221221
Processing date: 20221222
Processing date: 20221223
Skipping 20221223, missing file.
Processing date: 